# Ordinary Least Squares(OLS) vs Decision-Focused 

This notebook uses **gurobiPy** to solve the optimization problems for:

1. The **ground-truth** decisions $x^*$ by minimizing $x^2 - yx$ with bounds $0 \le x \le 1$.
2. Decision-focused learning: optimize $b_0,b_1$ for $\sum_t [(b_0+b_1 z_t)^2 - y_t(b_0+b_1 z_t)]$; for evaluation we clip $x=b_0+b_1 z$ to $[0,1]$.

For the **classical regression** baseline, we use **scikit-learn** OLS with intercept to fit $x^*$ from $z$.

**Evaluation** uses the cost: $\sum_t (x_t^2 - y_t x_t)$. Lower is better.


## 0) Imports and data

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

try:
    import gurobipy as gp
    from gurobipy import GRB
    GUROBI_OK = True
except Exception as e:
    GUROBI_OK = False
    print("gurobipy is not available in this environment. "
          "Install and ensure a valid Gurobi license to run the optimization cells.")
    print("Error:", e)

# Data
z = np.array([1, 4, 5, 10], dtype=float)
y = np.array([0.5, 2.33, 2.43, 1.88], dtype=float)

## 1) Helper functions

In [2]:
def clip01(x: np.ndarray) -> np.ndarray:
    return np.clip(x, 0.0, 1.0)

def total_cost(x: np.ndarray, y: np.ndarray) -> float:
    return float(np.sum(x**2 - y * x))

## 2) Ground-truth $x^*$ with Gurobi (Quadratic Programming(QP))

In [7]:
if not GUROBI_OK:
    raise RuntimeError("gurobipy not available. Please install and license Gurobi to run this cell.")

T = len(z)
m = gp.Model("xstar_qp")
m.Params.OutputFlag = 0

x_vars = m.addVars(T, lb=0.0, ub=1.0, name="x")

obj = gp.QuadExpr()
for t in range(T):
    obj.add(x_vars[t] * x_vars[t])
    obj.add(- y[t] * x_vars[t])

m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

x_star = np.array([x_vars[t].X for t in range(T)], dtype=float)
print("x* from Gurobi:", x_star)
print("Total cost (x*):", total_cost(x_star, y))

x* from Gurobi: [0.25 1.   1.   0.94]
Total cost (x*): -3.7061


## 3) Classical regression (OLS) — scikit-learn

## 4) Decision-Focused — Gurobi QP for $(b_0,b_1)$

Solve
$
\min_{b_0,b_1} \sum_t \big[(b_0 + b_1 z_t)^2 - y_t(b_0 + b_1 z_t)\big].
$
This yields a linear rule $x=b_0+b_1 z$. We clip $x$ to $[0,1]$ for evaluation.


## 5) Comparison the cost

## 6) Calculate the regret

## 7) Plot and save the figure